In [16]:
import caffe
import numpy as np
import shutil

In [17]:
root = "/media/vanilla/Fun/zhongxing/Test/"
model = "/media/vanilla/Fun/zhongxing/banben/TestModel.caffemodel"
prototxt = "/media/vanilla/Fun/zhongxing/banben/TestModel.prototxt"
caffe.set_mode_gpu()
net = caffe.Net(prototxt, model, caffe.TEST)

In [76]:
class Pruner(object):
    def __init__(self, net):
        self._net = net
        self.conv_data = {}
        self.candi  = ['conv3_1_1b','conv4_1_1b','conv5_1_1b']
        
    def _prune_res(self, conv_param, del_kernels=None, not_del_filters=False, ind=0):
        weight, bias = conv_param
        weight = weight.data
        bias = bias.data
        origin_channels = weight.shape[0]
        mask = self.candi[ind]
        print('weight start:',weight.shape)
#         print(del_kernels)
        cnt = 0
        del_filters = []
        maxx = self.conv_data[mask][0].shape[0]
#         print(self.conv_data[mask][0].shape,origin_channels,maxx)
#         print(self.conv_data[mask][0].shape)
        
        # delete filters
        if not not_del_filters:
            abs_mean = np.abs(weight).mean(axis=(1,2,3))
            for index,v in enumerate(abs_mean):
#                 if v < 5e-1 and cnt < origin_channels-maxx:
#                     del_filters.append(index)
#                     cnt += 1
#                     print(cnt)
#                 abs_mean = np.abs(weight).mean(axis=(1,2,3))
#                 a = sorted(list(abs_mean))
#                 print(list(abs_mean).index(a[:origin_channels-maxx]))
                del_filters = np.where(abs_mean < 1e-22)[0]
                del_filters = del_filters[:origin_channels-maxx]
            print(del_filters)
            weight = np.delete(weight, del_filters, axis=0)
#             print(weight.shape)
            bias = np.delete(bias, del_filters, axis=0)
        else:
            del_filters = np.array([])
        
        # delete kernels
        if del_kernels is not None:
            weight = np.delete(weight, del_kernels, axis=1)
            print(weight.shape)
#         else:
#             abs_mean = np.abs(weight).mean(axis=(1,2,3))
#             for index,v in enumerate(abs_mean):
#     #                 if v < 5e-1 and cnt < origin_channels-maxx:
#     #                     del_filters.append(index)
#     #                     cnt += 1
#     #                     print(cnt)
#     #             abs_mean = np.abs(weight).mean(axis=(1,2,3))
#     #             a = sorted(list(abs_mean))
#     #                 print(list(abs_mean).index(a[:origin_channels-maxx]))
#                 del_filters = np.where(abs_mean < 1e-22)[0]
#                 del_filters = del_filters[:origin_channels-maxx]
#             weight = np.delete(weight, del_filters, axis=1)
    #             print(weight.shape)
    #         bias = np.delete(bias, del_filters, axis=1)


            
        print('weight final:',weight.shape)
            
        return weight, bias, del_filters, origin_channels
        
        
        
    def _prune(self, conv_param, del_kernels=None, not_del_filters=False, layer='conv'):
        weight, bias = conv_param
        weight = weight.data
        bias = bias.data
        origin_channels = weight.shape[0]
#         print("del_kernels:",del_kernels)
        print('weight start:',weight.shape)
        
        # delete filters
        if not not_del_filters:
            abs_mean = np.abs(weight).mean(axis=(1,2,3))
            del_filters = np.where(abs_mean < 1e-32)[0]
            print(type(del_filters), len(del_filters))
            print(min(abs_mean))
            weight = np.delete(weight, del_filters, axis=0)
            bias = np.delete(bias, del_filters, axis=0)
        else:
            del_filters = np.array([])
        
        # delete kernels
        if del_kernels is not None:
            if layer == "conv":
#                 print(weight)
                weight = np.delete(weight, del_kernels, axis=1)
#                 print(weight.shape)
            else:
                weight = np.reshape(weight,(160,1536,9,9))
                weight = np.delete(weight, del_kernels, axis=1)
#                 print(weight.shape)
                weight = np.reshape(weight, (160,-1))
        print('weight final:',weight.shape)
            
        return weight, bias, del_filters, origin_channels
    
    def prune_conv(self, name, bottom=None):
        if bottom is None:
            self.conv_data[name] = self._prune(self._net.params[name])
        else:
            self.conv_data[name] = self._prune(self._net.params[name], self.conv_data[bottom][2])
            
    def prune_resual(self, name, bottom=None, ind=0,not_del_filters=True):
        if bottom is None:
            self.conv_data[name] = self._prune_res(self._net.params[name],ind=ind,not_del_filters=not_del_filters)
        else:
            self.conv_data[name] = self._prune_res(self._net.params[name], self.conv_data[bottom][2],not_del_filters=False,ind=ind)
            
    def prune_concat(self, name, bottoms,layer="conv"):
        if layer == 'conv':
            offsets = [0] + [self.conv_data[b][3] for b in bottoms]
#             print(offsets)
            for i in range(1, len(offsets)):
                offsets[i] += offsets[i-1]
#             print(offsets)
    #         for v in bottoms:
    #             print(self.conv_data[v][2])
            del_filters = [self.conv_data[b][2] + offsets[i] for i, b in enumerate(bottoms)]
#             print(del_filters)
            del_filters_new = np.concatenate(del_filters)
            a = self._net.params[name][0].data.shape
#             print("hahaha:",del_filters_new,a)
            if name == 'conv5_1_1b':
                self.conv_data[name] = self._prune(self._net.params[name], del_filters_new, not_del_filters=False)
            else:
                self.conv_data[name] = self._prune(self._net.params[name], del_filters_new, not_del_filters=True)
        else:
            offsets = [0] + [self.conv_data[b][3] for b in bottoms]
#             print(offsets)
            for i in range(1, len(offsets)):
                offsets[i] += offsets[i-1]
            print(offsets)
    #         for v in bottoms:
    #             print(self.conv_data[v][2])
            del_filters = [self.conv_data[b][2] + offsets[i] for i, b in enumerate(bottoms)]
#             print(del_filters)
            del_filters_new = np.concatenate(del_filters)
            a = self._net.params[name][0].data.shape
#             print("hahaha:",del_filters_new,a)
            if name == 'conv5_1_1b':
                self.conv_data[name] = self._prune(self._net.params[name], del_filters_new, not_del_filters=True)
            else:
                self.conv_data[name] = self._prune(self._net.params[name], del_filters, not_del_filters=True, layer='fc')
    
    def prune_sum(self, name, bottoms,layer="conv"):
        out = 0
        del_filters = []
        weight, bias = self._net.params[name]
        weight = weight.data
        bias = bias.data
        origin_channels = weight.shape[0]
        print('weight start :', weight.shape)
        cha = self.conv_data[bottoms[0]][0].shape[1]
        for i in bottoms:
            out += self.conv_data[i][0][:,:cha,:,:]
        weight = out
        print('weight finally :', weight.shape)
        self.conv_data[name] = (weight, bias, del_filters, origin_channels)
    
    def prune_concat_fc(self, name, bottoms,layer="conv"):
        out = 0
        cnt = 0
        del_filters = [ ]
        weight, bias = self._net.params[name]
        weight = weight.data
        bias = bias.data
        origin_channels = weight.shape[0]
        print('weight start :', weight.shape)
        cha = self.conv_data[bottoms[0]][0].shape[1]
        origin_channels = 512
        for i in bottoms:
            if cnt == 2:
                break
            out += self.conv_data[i][0][:,:cha,:,:]
            cnt += 1
        self.conv_data['mask'] = (out, bias, del_filters, origin_channels)   
        bottoms.remove("conv5_6_2")
        bottoms.remove("conv5_6_1")
        bottoms.append("mask")
        offsets = [0] + [self.conv_data[b][3] for b in bottoms]
#             print(offsets)
        for i in range(1, len(offsets)):
            offsets[i] += offsets[i-1]
        print(offsets)
        for v in bottoms:
            print(self.conv_data[v][2])
        del_filters = [np.asarray(self.conv_data[b][2]) + offsets[i] for i, b in enumerate(bottoms)]
        print(del_filters)
        del_filters_new = np.concatenate(del_filters)
        print(del_filters_new)
#         a = self._net.params[name][0].data.shape
#             print("hahaha:",del_filters_new,a)
        if name == 'conv5_1_1b':
            self.conv_data[name] = self._prune(self._net.params[name], del_filters_new, not_del_filters=True)
        else:
            self.conv_data[name] = self._prune(self._net.params[name], del_filters_new, not_del_filters=True, layer='fc')
            
            
            
            
#         weight = np.reshape(weight,(160,1536,9,9))
#         weight = np.delete(weight, del_kernels, axis=1)
#         #                 print(weight.shape)
#         weight = np.reshape(weight, (160,-1))
#         weight = out
#         print('weight finally :', weight.shape)
#         self.conv_data[name] = (weight, bias, del_filters, origin_channels)    
    
    
    
    
    def save(self, new_model, output_weights):
        net2 = caffe.Net(new_model, caffe.TEST)
#         print(self.conv_data.keys())
        for key in net2.params.keys():
            print(key)
            if key in self.conv_data:
                print(self.conv_data[key][0].shape)
                net2.params[key][0].data[...] = self.conv_data[key][0]
                net2.params[key][1].data[...] = self.conv_data[key][1]
            else:
#                 print(self.conv_data[key][0].shape,net2.params[key][0].shape)
                net2.params[key][0].data[...] = net.params[key][0].data
                net2.params[key][1].data[...] = net.params[key][1].data
        net2.save(output_weights)

In [77]:
pruner = Pruner(net)

In [78]:

pruner.prune_conv("conv1_1_1")
pruner.prune_conv("conv1_2_1")
pruner.prune_conv("conv1_2_2", "conv1_2_1")
pruner.prune_conv("conv1_3_1")
pruner.prune_conv("conv1_3_2", "conv1_3_1")
pruner.prune_conv("conv1_3_3", "conv1_3_2")
pruner.prune_concat("conv2_1", ("conv1_1_1", "conv1_2_2", "conv1_3_3"))
pruner.prune_conv("conv2_2", "conv2_1")
pruner.prune_conv("conv2_3", "conv2_2")
pruner.prune_conv("conv2_4", "conv2_3")
pruner.prune_conv("conv2_5", "conv2_4")
pruner.prune_conv("conv2_6", "conv2_5")
pruner.prune_conv("conv2_7", "conv2_6")
pruner.prune_conv("conv2_8", "conv2_7")

weight start: (32, 3, 3, 3)
<class 'numpy.ndarray'> 1
7.071002e-33
weight final: (31, 3, 3, 3)
weight start: (32, 3, 3, 3)
<class 'numpy.ndarray'> 3
5.3326602e-33
weight final: (29, 3, 3, 3)
weight start: (32, 32, 3, 3)
<class 'numpy.ndarray'> 1
7.738279e-33
weight final: (31, 29, 3, 3)
weight start: (32, 3, 3, 3)
<class 'numpy.ndarray'> 2
5.7964817e-33
weight final: (30, 3, 3, 3)
weight start: (32, 32, 3, 3)
<class 'numpy.ndarray'> 4
2.5248304e-33
weight final: (28, 30, 3, 3)
weight start: (32, 32, 3, 3)
<class 'numpy.ndarray'> 5
7.3803137e-34
weight final: (27, 28, 3, 3)
weight start: (64, 96, 3, 3)
weight final: (64, 89, 3, 3)
weight start: (64, 64, 3, 3)
<class 'numpy.ndarray'> 0
0.004236547
weight final: (64, 64, 3, 3)
weight start: (64, 64, 3, 3)
<class 'numpy.ndarray'> 1
3.7985343e-33
weight final: (63, 64, 3, 3)
weight start: (64, 64, 3, 3)
<class 'numpy.ndarray'> 0
0.01365149
weight final: (64, 63, 3, 3)
weight start: (64, 64, 3, 3)
<class 'numpy.ndarray'> 0
0.019319572
weight

/home/vanilla/.conda/envs/testcaffe/lib/python3.5/site-packages/ipykernel_launcher.py:92: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


In [79]:
pruner.prune_concat("conv3_1_1", ("conv2_2", "conv2_4", "conv2_6", "conv2_8"))
pruner.prune_concat("conv3_1_1b", ("conv2_2", "conv2_4", "conv2_6", "conv2_8"))

weight start: (128, 256, 3, 3)
weight final: (128, 252, 3, 3)
weight start: (128, 256, 1, 1)
weight final: (128, 252, 1, 1)


In [80]:
pruner.prune_resual("conv3_1_2", "conv3_1_1",ind=0)
pruner.prune_resual("conv3_2_1","conv3_1_1b",ind=0)
pruner.prune_resual("conv3_2_2", "conv3_2_1",ind=0)
pruner.prune_resual("conv3_3_1","conv3_1_1b",ind=0)
pruner.prune_resual("conv3_3_2", "conv3_3_1",ind=0)
pruner.prune_resual("conv3_4_1","conv3_1_1b",ind=0)
pruner.prune_resual("conv3_4_2", "conv3_4_1",ind=0)
pruner.prune_resual("conv3_5_1","conv3_1_1b",ind=0)
pruner.prune_resual("conv3_5_2", "conv3_5_1",ind=0)
pruner.prune_resual("conv3_6_1","conv3_1_1b",ind=0)
pruner.prune_resual("conv3_6_2", "conv3_6_1",ind=0)
pruner.prune_concat("conv4_1_1", ("conv3_2_2", "conv3_4_2", "conv3_6_2"))
pruner.prune_concat("conv4_1_1b", ("conv3_2_2", "conv3_4_2", "conv3_6_2"))

weight start: (128, 128, 3, 3)
[]
(128, 128, 3, 3)
weight final: (128, 128, 3, 3)
weight start: (128, 128, 3, 3)
[]
(128, 128, 3, 3)
weight final: (128, 128, 3, 3)
weight start: (128, 128, 3, 3)
[]
(128, 128, 3, 3)
weight final: (128, 128, 3, 3)
weight start: (128, 128, 3, 3)
[]
(128, 128, 3, 3)
weight final: (128, 128, 3, 3)
weight start: (128, 128, 3, 3)
[]
(128, 128, 3, 3)
weight final: (128, 128, 3, 3)
weight start: (128, 128, 3, 3)
[]
(128, 128, 3, 3)
weight final: (128, 128, 3, 3)
weight start: (128, 128, 3, 3)
[]
(128, 128, 3, 3)
weight final: (128, 128, 3, 3)
weight start: (128, 128, 3, 3)
[]
(128, 128, 3, 3)
weight final: (128, 128, 3, 3)
weight start: (128, 128, 3, 3)
[]
(128, 128, 3, 3)
weight final: (128, 128, 3, 3)
weight start: (128, 128, 3, 3)
[]
(128, 128, 3, 3)
weight final: (128, 128, 3, 3)
weight start: (128, 128, 3, 3)
[]
(128, 128, 3, 3)
weight final: (128, 128, 3, 3)
weight start: (256, 384, 3, 3)
weight final: (256, 384, 3, 3)
weight start: (256, 384, 1, 1)
weigh

/home/vanilla/.conda/envs/testcaffe/lib/python3.5/site-packages/ipykernel_launcher.py:43: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


In [81]:
pruner.prune_resual("conv4_1_2", "conv4_1_1",ind=1)
pruner.prune_resual("conv4_2_1","conv4_1_1b",ind=1)
pruner.prune_resual("conv4_2_2", "conv4_2_1",ind=1)
pruner.prune_resual("conv4_3_1","conv4_1_1b",ind=1)
pruner.prune_resual("conv4_3_2", "conv4_3_1",ind=1)
pruner.prune_resual("conv4_4_1","conv4_1_1b",ind=1)
pruner.prune_resual("conv4_4_2", "conv4_4_1",ind=1)
pruner.prune_resual("conv4_5_1","conv4_1_1b",ind=1)
pruner.prune_resual("conv4_5_2", "conv4_5_1",ind=1)
pruner.prune_resual("conv4_6_1","conv4_1_1b",ind=1)
pruner.prune_resual("conv4_6_2", "conv4_6_1",ind=1)
pruner.prune_concat("conv5_1_1", ("conv4_2_2", "conv4_4_2", "conv4_6_2"))
pruner.prune_concat("conv5_1_1b", ("conv4_2_2", "conv4_4_2", "conv4_6_2"))

weight start: (256, 256, 3, 3)
[]
(256, 256, 3, 3)
weight final: (256, 256, 3, 3)
weight start: (256, 256, 3, 3)
[]
(256, 256, 3, 3)
weight final: (256, 256, 3, 3)
weight start: (256, 256, 3, 3)
[]
(256, 256, 3, 3)
weight final: (256, 256, 3, 3)
weight start: (256, 256, 3, 3)
[]
(256, 256, 3, 3)
weight final: (256, 256, 3, 3)
weight start: (256, 256, 3, 3)
[]
(256, 256, 3, 3)
weight final: (256, 256, 3, 3)
weight start: (256, 256, 3, 3)
[]
(256, 256, 3, 3)
weight final: (256, 256, 3, 3)
weight start: (256, 256, 3, 3)
[]
(256, 256, 3, 3)
weight final: (256, 256, 3, 3)
weight start: (256, 256, 3, 3)
[]
(256, 256, 3, 3)
weight final: (256, 256, 3, 3)
weight start: (256, 256, 3, 3)
[]
(256, 256, 3, 3)
weight final: (256, 256, 3, 3)
weight start: (256, 256, 3, 3)
[]
(256, 256, 3, 3)
weight final: (256, 256, 3, 3)
weight start: (256, 256, 3, 3)
[]
(256, 256, 3, 3)
weight final: (256, 256, 3, 3)
weight start: (512, 768, 1, 1)
weight final: (512, 768, 1, 1)
weight start: (512, 768, 1, 1)
<clas

/home/vanilla/.conda/envs/testcaffe/lib/python3.5/site-packages/ipykernel_launcher.py:43: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


In [82]:
pruner.prune_resual("conv5_1_2","conv5_1_1",ind=2)
pruner.prune_sum("conv5_2_1",("conv5_1_2","conv5_1_1b"))
pruner.prune_resual("conv5_2_2","conv5_2_1",ind=2)
pruner.prune_sum("conv5_3_1",("conv5_2_2","conv5_2_1"))
pruner.prune_resual("conv5_3_2","conv5_3_1",ind=2)
pruner.prune_sum("conv5_4_1",("conv5_3_2","conv5_3_1"))
pruner.prune_resual("conv5_4_2","conv5_4_1",ind=2)
pruner.prune_sum("conv5_5_1",("conv5_4_2","conv5_4_1"))
pruner.prune_resual("conv5_5_2","conv5_5_1",ind=2)
pruner.prune_sum("conv5_6_1",("conv5_5_2","conv5_5_1"))
pruner.prune_resual("conv5_6_2","conv5_6_1",ind=2)


weight start: (512, 512, 1, 1)
[ 97 178]
(510, 512, 1, 1)
weight final: (510, 512, 1, 1)
weight start : (512, 512, 1, 1)
weight finally : (510, 512, 1, 1)
weight start: (512, 512, 1, 1)
[2 9]
(510, 512, 1, 1)
weight final: (510, 512, 1, 1)
weight start : (512, 512, 1, 1)
weight finally : (510, 512, 1, 1)
weight start: (512, 512, 1, 1)
[0 6]
(510, 512, 1, 1)
weight final: (510, 512, 1, 1)
weight start : (512, 512, 1, 1)
weight finally : (510, 512, 1, 1)
weight start: (512, 512, 1, 1)
[ 0 97]
(510, 512, 1, 1)
weight final: (510, 512, 1, 1)
weight start : (512, 512, 1, 1)
weight finally : (510, 512, 1, 1)
weight start: (512, 512, 1, 1)
[13 27]
(510, 512, 1, 1)
weight final: (510, 512, 1, 1)
weight start : (512, 512, 1, 1)
weight finally : (510, 512, 1, 1)
weight start: (512, 512, 1, 1)
[ 13 152]
(510, 512, 1, 1)
weight final: (510, 512, 1, 1)


/home/vanilla/.conda/envs/testcaffe/lib/python3.5/site-packages/ipykernel_launcher.py:43: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


In [83]:
pruner.prune_concat_fc("fc5_1", ["conv5_6_2", "conv5_6_1","conv5_3_2","conv5_5_1"], layer="fc")
# pruner.prune_concat("conv6_1_1b", ("conv5_2_2", "conv5_4_2", "conv5_6_2"))

weight start : (160, 124416)
[0, 512, 1024, 1536]
[0 6]
[]
[]
[array([0, 6]), array([], dtype=float64), array([], dtype=float64)]
[0. 6.]
weight start: (160, 124416)
weight final: (160, 124254)


/home/vanilla/.conda/envs/testcaffe/lib/python3.5/site-packages/ipykernel_launcher.py:96: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


In [84]:
[(k, v[0].shape[0]) for k, v in pruner.conv_data.items() if v[0] is not None]

[('fc5_1', 160),
 ('conv4_6_2', 256),
 ('conv5_5_1', 510),
 ('conv2_8', 62),
 ('conv5_3_1', 510),
 ('conv3_5_1', 128),
 ('mask', 510),
 ('conv4_3_1', 256),
 ('conv5_4_2', 510),
 ('conv4_1_2', 256),
 ('conv3_6_1', 128),
 ('conv3_4_2', 128),
 ('conv5_2_1', 510),
 ('conv1_2_1', 29),
 ('conv2_5', 64),
 ('conv3_1_1b', 128),
 ('conv4_2_2', 256),
 ('conv1_3_2', 28),
 ('conv4_2_1', 256),
 ('conv5_4_1', 510),
 ('conv4_5_1', 256),
 ('conv5_3_2', 510),
 ('conv3_3_1', 128),
 ('conv3_2_1', 128),
 ('conv4_5_2', 256),
 ('conv3_1_1', 128),
 ('conv2_4', 64),
 ('conv4_4_2', 256),
 ('conv5_6_2', 510),
 ('conv3_6_2', 128),
 ('conv4_1_1', 256),
 ('conv5_6_1', 510),
 ('conv5_1_1', 512),
 ('conv4_1_1b', 256),
 ('conv1_2_2', 31),
 ('conv3_1_2', 128),
 ('conv3_5_2', 128),
 ('conv3_4_1', 128),
 ('conv5_5_2', 510),
 ('conv2_2', 64),
 ('conv5_2_2', 510),
 ('conv5_1_2', 510),
 ('conv1_3_3', 27),
 ('conv1_1_1', 31),
 ('conv5_1_1b', 510),
 ('conv4_6_1', 256),
 ('conv4_3_2', 256),
 ('conv3_2_2', 128),
 ('conv2_7', 64

In [85]:
def get_prototxt(pk, pro_n):  # 复制原来的prototxt,并修改修剪层的num_output,这一段代码有点绕,有空的话优化为几个单独的函数或者弄个类
    with open(pro_n, "r") as p:
        lines = p.readlines()
    k = 0
    with open(pro_n, "w") as p:
        while k < len(lines):  # 遍历所有的lines,此处不宜用for.
#             print("lines[k]:",lines[k])
            if 'name:' in lines[k]:
                print("lines[k].split:",lines[k].split('"')[1])
#                 print(pk.keys())
                l_name = lines[k].split('"')[1]  # 获取layer name
                if l_name in pk.keys():  # 如果name在待修剪层中,则需要修改,下面进入一个找channel的循环块.
                    while True:
                        if "num_output:" in lines[k]:
                            channel_n = "    num_output: " + str(len(pk[l_name][0])) + "\n"
                            print(channel_n)
                            p.write(channel_n)
                            k = k + 1
                            break
                        else:
                            p.write(lines[k])
                            k = k + 1
                else:  # name不在待修剪层中,直接copy行
                    p.write(lines[k])
                    k = k + 1
 
            else:
                p.write(lines[k])
                k = k + 1
    print("deploy_rebirth_prune.prototxt已写好")

In [86]:
pro_n = root + "TestModel.prototxt"
shutil.copyfile(prototxt, pro_n)
get_prototxt(pruner.conv_data, pro_n)

lines[k].split: TestModel
lines[k].split: conv1_1_1
    num_output: 31

lines[k].split: conv1_1_1_relu
lines[k].split: conv1_2_1
    num_output: 29

lines[k].split: conv1_2_1_relu
lines[k].split: conv1_2_2
    num_output: 31

lines[k].split: conv1_2_2_relu
lines[k].split: conv1_3_1
    num_output: 30

lines[k].split: conv1_3_1_relu
lines[k].split: conv1_3_2
    num_output: 28

lines[k].split: conv1_3_2_relu
lines[k].split: conv1_3_3
    num_output: 27

lines[k].split: conv1_3_3_relu
lines[k].split: feature1
lines[k].split: conv2_1
    num_output: 64

lines[k].split: conv2_1_relu
lines[k].split: conv2_2
    num_output: 64

lines[k].split: conv2_2_relu
lines[k].split: conv2_3
    num_output: 63

lines[k].split: conv2_3_relu
lines[k].split: conv2_4
    num_output: 64

lines[k].split: conv2_4_relu
lines[k].split: conv2_5
    num_output: 64

lines[k].split: conv2_5_relu
lines[k].split: conv2_6
    num_output: 62

lines[k].split: conv2_6_relu
lines[k].split: conv2_7
    num_output: 64

lines

In [87]:
# You should modify the number of channels in new prototxt before save
pruner.save(pro_n, "/media/vanilla/Fun/zhongxing/Test/TestModel.caffemodel")

conv1_1_1
(31, 3, 3, 3)
conv1_2_1
(29, 3, 3, 3)
conv1_2_2
(31, 29, 3, 3)
conv1_3_1
(30, 3, 3, 3)
conv1_3_2
(28, 30, 3, 3)
conv1_3_3
(27, 28, 3, 3)
conv2_1
(64, 89, 3, 3)
conv2_2
(64, 64, 3, 3)
conv2_3
(63, 64, 3, 3)
conv2_4
(64, 63, 3, 3)
conv2_5
(64, 64, 3, 3)
conv2_6
(62, 64, 3, 3)
conv2_7
(64, 62, 3, 3)
conv2_8
(62, 64, 3, 3)
conv3_1_1b
(128, 252, 1, 1)
conv3_1_1
(128, 252, 3, 3)
conv3_1_2
(128, 128, 3, 3)
conv3_2_1
(128, 128, 3, 3)
conv3_2_2
(128, 128, 3, 3)
conv3_3_1
(128, 128, 3, 3)
conv3_3_2
(128, 128, 3, 3)
conv3_4_1
(128, 128, 3, 3)
conv3_4_2
(128, 128, 3, 3)
conv3_5_1
(128, 128, 3, 3)
conv3_5_2
(128, 128, 3, 3)
conv3_6_1
(128, 128, 3, 3)
conv3_6_2
(128, 128, 3, 3)
conv4_1_1b
(256, 384, 1, 1)
conv4_1_1
(256, 384, 3, 3)
conv4_1_2
(256, 256, 3, 3)
conv4_2_1
(256, 256, 3, 3)
conv4_2_2
(256, 256, 3, 3)
conv4_3_1
(256, 256, 3, 3)
conv4_3_2
(256, 256, 3, 3)
conv4_4_1
(256, 256, 3, 3)
conv4_4_2
(256, 256, 3, 3)
conv4_5_1
(256, 256, 3, 3)
conv4_5_2
(256, 256, 3, 3)
conv4_6_1
(256, 256

ValueError: could not broadcast input array from shape (510,512,1,1) into shape (510,510,1,1)